IMPORTING LIBRARIES

In [1]:
from stable_baselines3 import DQN, PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from sb3_contrib.common.wrappers import ActionMasker
from deterioration_model import bridgedeterioration
import numpy as np
import gymnasium as gym
import torch as th

In [2]:
import logging
logging.basicConfig(level=logging.DEBUG)  # or logging.INFO

In [3]:
env = bridgedeterioration(df_path='bridge_data.csv')

# Wrap the environment with DummyVecEnv
env = DummyVecEnv([lambda: env])

# Normalize the observations and rewards
env = VecNormalize(env, norm_obs=True, norm_reward= True, clip_obs= float('inf'))

In [4]:
env_unwrapped = env.envs[0]
obs = env_unwrapped.reset()
obs = obs[0]
np.round(obs[:,0])

array([ 4000.,  8000.,  8000.,  4000.,  4000.,  4000.,  4000.,  4000.,
        4000.,  4000.,  4000.,  4000.,  4000.,  4000., 20000.],
      dtype=float32)

In [5]:
policy_kwargs = dict(activation_fn=th.nn.ReLU, net_arch=[128, 128])
env.reset()
model = PPO("MlpPolicy", env, n_epochs = 10, learning_rate=0.01, batch_size = 32, gamma = 0.99, policy_kwargs= policy_kwargs,verbose=2)
model.learn(total_timesteps=200, log_interval=4)

Using cpu device


In [6]:
env.training = False # Setting env to unnormalized state
env.norm_reward = False

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100, deterministic= True)
print(f"Mean reward: {mean_reward} ± {std_reward}")

d:\anaconda3\envs\rllib\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward: -8255160.003173828 ± 0.0
